In [1]:
! nvidia-smi

Tue May 19 13:26:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
from __future__ import print_function
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from PIL import Image
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import cv2
import pkgutil
import importlib
import os
%matplotlib inline
%config IPCompleter.greedy=True
%reload_ext autoreload
import seaborn as sns
import io
import skimage
from skimage.transform import resize
from itertools import groupby
from tqdm.auto import tqdm
from pathlib import Path
from time import time
from datetime import datetime 
import gc
sns.set()
from zipfile import ZipFile
import zipfile

# install "apex" model for mixed precision training if required.. this is needed for LRFinder
if pkgutil.find_loader("apex") is None:
   print("****apex module does not exist..hence installing*****")
   !pip install -U git+https://www.github.com/NVIDIA/apex --no-cache-dir

import albumentations
print(albumentations.__version__) # check version

# This is to install and use albumentations latest package v0.4.5
album_version_needed = "0.4.5"
if albumentations.__version__ != album_version_needed:
  !pip install albumentations==0.4.5   # 0.4.5 is the latest albumentation version
print('Pytorch version:', torch.__version__)

!pip install kornia
from kornia.losses import SSIM

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


****apex module does not exist..hence installing*****
  Cloning https://www.github.com/NVIDIA/apex to /tmp/pip-req-build-sczw6apz
  Running command git clone -q https://www.github.com/NVIDIA/apex /tmp/pip-req-build-sczw6apz
  Running command git submodule update --init --recursive -q
  Created wheel for apex: filename=apex-0.1-cp36-none-any.whl size=177174 sha256=60867b2e7114f8ef70a3d69a2d146079a207a63cb772d3f209c05f9b11d1d30f
  Stored in directory: /tmp/pip-ephem-wheel-cache-4rbjm5p4/wheels/08/3e/36/a75a9914e497fe42598f9dbe67496b2b300a8851f4d396a285
Successfully built apex
0.1.12
     |████████████████████████████████| 122kB 2.8MB/s 
     |████████████████████████████████| 634kB 8.7MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=64378 sha256=90bbc29906adf5c3dc5486f057f1ca626fbe79f14cc2b56a24299a0d99d93d12
  Stored in directory: /root/.cache/pip/wheels/f0/a0/61/e50f93165a5ec7e7f5d65064e513239505bc4c06d2289557d3
  Created wheel for imgaug: 

Pytorch version: 1.5.0+cu101
     |████████████████████████████████| 163kB 2.8MB/s 


In [4]:
os.mkdir('/content/sample_data/FG_BG')
os.mkdir('/content/sample_data/Mask')
os.mkdir('/content/sample_data/Depth')

FG_BG_zip =  ZipFile("/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Mask_Depth/FG_BG_400K.zip", 'r')
Mask_zip  = ZipFile("/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Mask_Depth/FG_BG_Mask_400K.zip", 'r')
Depth_zip = ZipFile("/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Mask_Depth/FG_BG_Depth_400K.zip", 'r')
all_files_FG_BG = [info.filename for info in FG_BG_zip.infolist() if not info.is_dir()]
all_files_Mask  = [info.filename for info in Mask_zip.infolist() if not info.is_dir()]
all_files_Depth = [info.filename for info in Depth_zip.infolist() if not info.is_dir()]
print(len(all_files_FG_BG))
print(len(all_files_Mask))
print(len(all_files_Depth))

path_fg_bg  = '/content/sample_data/FG_BG/'
path_mask   = '/content/sample_data/Mask/' 
path_depth  = '/content/sample_data/Depth/'

t1 = time()
k = 0
for img_file in all_files_FG_BG[0:40000]:
    if img_file.endswith('jpg'):
        imgdata = FG_BG_zip.read(img_file)
        img = Image.open(io.BytesIO(imgdata))
        file_name = img_file.split('/')[-1].split('.')[0] + '.jpg'
        img.save(f'{path_fg_bg}{file_name}')
        k += 1
print('FG_BG Written:',k) 

k = 0
for img_file in all_files_Mask[0:]:
    if img_file.endswith('jpg'):
        imgdata = Mask_zip.read(img_file)
        img = Image.open(io.BytesIO(imgdata))
        file_name = img_file.split('/')[-1].split('.')[0] + '.jpg'
        img.save(f'{path_mask}{file_name}')
        k += 1
print('Masks Written:',k) 

k = 0
for img_file in all_files_Depth[0:]:
    if img_file.endswith('jpg'):
        imgdata = Depth_zip.read(img_file)
        img = Image.open(io.BytesIO(imgdata))
        file_name = img_file.split('/')[-1].split('.')[0] + '.jpg'
        img.save(f'{path_depth}{file_name}')
        k += 1
print('Depth Written:',k) 

!cp -r '/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Mask_Depth/BG_and_Its_Flip' '/content/sample_data/'
!cp '/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Mask_Depth/FG_BG_Filename_withflip_Logs.txt' '/content/sample_data/'

t2 = time()
print(f'Copy took {(t2-t1)/60:.5f} minutes')

FileNotFoundError: ignored

In [0]:
log_fg_bg = open('/content/sample_data/FG_BG_Filename_withflip_Logs.txt',"r")
content_list = log_fg_bg.readlines()
print('No: of records in log_fg_bg:',len(content_list))
print(content_list[0])
log_fg_bg.close()

data_root = Path('/content/sample_data')
f1, f2, f3, f4 = data_root/'FG_BG', data_root/'BG_and_Its_Flip', data_root/'Mask', data_root/'Depth'

In [0]:
print(len(list(f1.iterdir())))
print(len(list(f2.iterdir())))
print(len(list(f3.iterdir())))
print(len(list(f4.iterdir())))

In [0]:
bs   = 25
rows = 5
size = 256
prnt = 100
drpout = 0.2

In [0]:
gt_transform = transforms.Compose([transforms.Resize((256,256)),
                                      transforms.ToTensor(),])

In [0]:
bg_transform = transforms.Compose([#transforms.Resize((256,256)),
                                       transforms.ColorJitter(brightness =0.05,contrast=0.05,saturation=0.05,hue=0.05),
                                       transforms.ToTensor(),
                                       ])

In [0]:
fg_bg_transform = transforms.Compose([#transforms.Resize((256,256)),
                                       transforms.ColorJitter(brightness =0.05,contrast=0.05,saturation=0.05,hue=0.05),
                                       transforms.ToTensor(),
                                       ])

In [0]:
class MasterDataset(Dataset):
  def __init__(self,data_root,content_list, do_transform=True):
    self.f1_files     = list(f1.glob('*.jpg'))
    self.content_list = content_list
    self.do_transform    = do_transform

  def __len__(self):
    return len(self.f1_files)  

  def __getitem__(self,index):
    fg_bg_name   = self.f1_files[index].stem
    fg_bg_idx    = fg_bg_name.split('_')[-1]
    bg_name = '/content/sample_data/BG_and_Its_Flip/' + self.content_list[int(fg_bg_idx)-1].split(',')[1]
    mask_name    = '/content/sample_data/Mask/' + 'Img_fg_bg_mask' + str(fg_bg_idx) + '.jpg'
    depth_name   = '/content/sample_data/Mask/' + 'Img_fg_bg_' + str(fg_bg_idx) + '_depth.jpg'
    f1_image = Image.open(self.f1_files[index])  
    f2_image = Image.open(f'{bg_name}')
    f3_image = Image.open(f'{mask_name}')
    f4_image = Image.open(f'{depth_name}')

    if self.do_transform:
      f1_image = bg_transform(f1_image)
      f2_image = fg_bg_transform(f2_image)
      f3_image = (gt_transform(f3_image) > 0.8).float()
      f4_image = gt_transform(f4_image)

    return {'f1':f1_image, 'f2':f2_image, 'f3':f3_image, 'f4':f4_image}

In [0]:
train_ds = MasterDataset(data_root, content_list, train_transforms)

In [0]:
[(k, v.shape) for k, v in train_ds[0].items() if k != 'index']

In [0]:
train_dl = DataLoader(train_ds, batch_size = bs, shuffle = True, pin_memory = True)

In [0]:
sample = next(iter(train_dl))
type(sample)

In [0]:
[(k, v.shape) for k, v in sample.items()]

In [0]:
imgs = sample['f3']

In [0]:
def show(tensors, figsize= (10,10), *args, **kwargs):
  grid_tensor = torchvision.utils.make_grid(tensors, *args, **kwargs)  
  grid_image  = grid_tensor.permute(1, 2, 0)
  plt.figure(figsize = figsize)
  plt.imshow(grid_image)
  plt.xticks([])  
  plt.yticks([])
  plt.show()

In [0]:
def show_pred(tensors, *args, **kwargs):
  tensors = (tensors * std[None, : , None, None]) + mean[None, :, None, None]
  show(tensors, *args, **kwargs)

In [0]:
def draw_and_save(tensors, name, figsize=(20,20), *args, **kwargs):
  try:
    tensors = tensors.detach().cpu()
  except:
    pass
  grid_tensor = torchvision.utils.make_grid(tensors, *args, **kwargs)
  grid_image  = grid_tensor.permute(1, 2, 0)
  plt.figure(figsize = figsize)
  plt.imshow(grid_image)
  plt.xticks([])  
  plt.yticks([])

  plt.savefig(name, bbox_inches='tight')
  plt.show()

In [0]:
show(imgs, nrow=rows)

In [0]:
class DownResNet(nn.Module):
  def __init__(self,inchannels,outchannels):
    super(DownResNet, self).__init__()

    self.conv1  = nn.Conv2d(inchannels, outchannels, kernel_size=3, stride=2, padding=1, bias=False)
    self.conv12 = nn.Conv2d(outchannels, outchannels, kernel_size=1, stride=1, padding=0, bias=False)
    self.bn1 = nn.BatchNorm2d(outchannels)
    self.relu = nn.ReLU(inplace=False)

    self.conv2  = nn.Conv2d(outchannels, outchannels, kernel_size=3, stride=1, padding=1, bias=False)
    self.conv22 = nn.Conv2d(outchannels, outchannels, kernel_size=1, stride=1, padding=0, bias=False)
    self.bn2 = nn.BatchNorm2d(outchannels)

    self.conv1x1down = nn.Conv2d(inchannels, outchannels,kernel_size=1, stride=2, padding=0, bias=False)

  def forward(self,x):
    identity = self.conv1x1down(x)
    out = self.conv1(x)
    out = self.conv12(out)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.conv22(out)
    out = self.bn2(out)

    out += identity
    out = self.relu(out)

    return out 

In [0]:
class UpResNet(nn.Module):
  def __init__(self, inchannels, outchannels):
    super(UpResNet, self).__init__()

    self.conv1 = nn.Conv2d(outchannels, outchannels, kernel_size=3, stride=1, padding=1, bias=False)
    self.conv12 = nn.Conv2d(outchannels, outchannels, kernel_size=1, stride=1, padding=0, bias=False)
    self.bn1 = nn.BatchNorm2d(outchannels)
    self.relu = nn.ReLU(inplace = False)

    self.conv2 = nn.Conv2d(outchannels, outchannels, kernel_size=3, stride=1, padding=1, bias=False)
    self.conv22 = nn.Conv2d(outchannels, outchannels, kernel_size=1, stride=1, padding=0, bias=False)
    self.bn2 = nn.BatchNorm2d(outchannels)

    self.conv1x1down = nn.ConvTranspose2d(inchannels, outchannels, kernel_size=3, stride=1, padding=1, bias=False)

  def forward(self, x):
    identity = self.conv1x1down(x)
    out = self.conv1(identity)
    out = self.conv12(out)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.conv22(out)
    out = self.bn2(out)

  # out += x

    out = self.relu(out)  
    return out

In [0]:
class BGDPTH(nn.Module):
  def __init__(self):
    super(BGDPTH, self).__init__()

    self.convFirst = nn.Sequential(nn.Conv2d(6,64,kernel_size=7,stride=2,padding=3,bias=False),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=False))
    
    self.maxpool = nn.MaxPool2d(3, stride = 2, padding =1)

    self.rb1 = DownResNet(64,128)
    self.rb2 = DownResNet(128,256)
    self.rb3 = DownResNet(256,512)

    self.rb4 = UpResNet(512, 256)
    self.rb5 = UpResNet(256, 128)
    self.rb6 = UpResNet(128, 64)

    self.rb7 = UpResNet(64,64)

    self.rb8 = UpResNet(64,32)

    self.convLast = nn.Sequential(nn.Conv2d(32, 3, kernel_size=1, stride=1, padding=0, bias=False),)

    self.rb4D = UpResNet(512, 256)
    self.rb5D = UpResNet(256, 128) 
    self.rb6D = UpResNet(128, 64)

    self.rb7D = UpResNet(64, 64)

    self.rb8D = UpResNet(64, 32)

    self.convLastD = nn.Sequential(nn.Conv2d(32, 3, kernel_size=1, stride=1, padding=0, bias=False),)

  def forward(self, sample):
    f11 = sample['f1']
    f22 = sample['f2']
    f   = torch.cat([f11,f22], dim=1)

    out0 = self.convFirst(f)
    out1 = self.maxpool(out0)
    out2 = self.rb1(out1)
    out3 = self.rb2(out2)
    out4 = self.rb3(out3)

    out4 = nn.functional.interpolate(out4, scale_factor=2, mode='bilinear')

    out5 = self.rb4(out4)
    out5 += out3
    out5 = nn.functional.interpolate(out5, scale_factor=2, mode='bilinear')
    out6 = self.rb5(out5)
    out6 += out2 
    out6 = nn.functional.interpolate(out6, scale_factor=2, mode='bilinear')
    out7 = self.rb6(out6)
    out7 += out1
    out7 = nn.functional.interpolate(out7, scale_factor=2, mode='bilinear')
    out8 = self.rb7(out7)
    out8 += out0

    out9 = nn.functional.interpolate(out8, scale_factor=2, mode='bilinear')
    out9 = self.rb8(out9)

    outB = self.convLast(out9)

    out5D = self.rb4D(out4)
    out5D += out3
    out5D = nn.functional.interpolate(out5D, scale_factor=2, mode='bilinear')
    out6D = self.rb5D(out5D)
    out6D += out2
    out6D = nn.functional.interpolate(out6D, scale_factor=2, mode='bilinear')
    out7D = self.rb6D(out6D)
    out7D += out1
    out7D = nn.functional.interpolate(out7D, scale_factor=2, mode='bilinear')
    out8D = self.rb7D(out7D)
    out8D += out0

    out9D = nn.functional.interpolate(out8D, scale_factor=2, mode='bilinear')
    out9D = self.rb8D(out9D)

    outD  = self.convLastD(out9D)

    return outB, outD

In [0]:
def weights_init(ms):
  for m in ms.modules():
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1:
      m.weight.data = m.init.kaiming_normal_(m.weight.data, mode='fan_out',nonlinearity='relu')
    elif classname.find('ConvTranspose2d') != -1:
      m.weight.data = m.init.kaiming_normal_(m.weight.data, mode='fan_out',nonlinearity='relu')
    elif isinstance(m, nn.BatchNorm2d):
      nn.init.constant_(m.weight, 1)
      nn.init.constant_(m.bias,0)

In [0]:
#kernel_size = 3
#criterion = SSIM(kernel_size, reduction = 'mean)
criterion = nn.BCEWithLogitsLoss()

In [0]:
model    = BGDPTH()
use_cuda = torch.cuda.is_available()
model.device = torch.device("cuda" if use_cuda else "cpu")
model.to(model.device)

In [0]:
from torchsummary import summary
summary(model, input_size=(3,192,192))

In [0]:
optim = torch.optim.SGD(model.parameters(),lr=0.001, momentum = 0.9, weight_decay = 1e-5)

In [0]:
def train(model, criterion, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  for batch_idx, data in enumerate(pbar):
    data['f1'] = data['f1'].to(device)
    data['f2'] = data['f2'].to(device)
    data['f3'] = data['f3'].to(device)
    data['f4'] = data['f4'].to(device)    

    optimizer.zero_grad()
    output = model(data)

 # breakpoint()

    loss1 = criterion(output[0], data['f3'])
    loss2 = criterion(output[1], data['f4'])
    loss  = 2*loss1 + loss2

    pbar.set_description(desc = f'loss={loss.item()} l1={loss1.item()} l2={loss2.item()}')   
    loss.backward()
    optimizer.step()

    if batch_idx % prnt == 0:
      #show_pred(output.detach().cpu(), nrow =2) # nrow = sqrt-bs
      draw_and_save(output[0].detach().cpu(), f'plots/{epoch}_{batch-idx}_BR_{loss1.item()}.jpg')
      draw_and_save(output[1].detach().cpu(), f'plots/{epoch}_{batch-idx}_DR_{loss2.item()}.jpg')
      draw_and_save(data['f3'].detach().cpu(), f'plots/{epoch}_{batch-idx}_BG_{loss.item()}.jpg')
      draw_and_save(data['f4'].detach().cpu(), f'plots/{epoch}_{batch-idx}_DG_{loss.item()}.jpg')
      draw_and_save(data['f2'].detach().cpu(), f'plots/{epoch}_{batch-idx}_IN_{loss.item()}.jpg')

      #show(data['f2'].detach().cpu(), nrow=5)

    if batch_idx % 650 == 0:
      torch.save(model.state_dict(),PATH/f'{epoch}_{batch_idx}_{loss.item()}.pth')

In [0]:
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optim, step_size=2, gamma=0.1)

In [0]:
for epoch in range(1,2):
  train(model, criterion,device, train_dl, optim, epoch)
  #test(model, device, test_loader)
  scheduler.step()